In [1]:
import pickle
import numpy as np
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
#from galaxymodule.galaxy import Galaxy
import tree
import collections
#from galaxymodule import galaxy
import load

from analysis.cal_lambda import *
from temp.makegal import *

In [2]:
nout=782
Mcut = 1e10

cal_lambda_params = dict(npix_per_reff=5,
                        rscale=3.0,
                        method='ellip',
                        n_pseudo=1,
                        verbose=False,
                        voronoi=None,#voronoi_dict,
                        mge_interpol = True)
mk_gal_params = dict()

# Load data
import tree


s = load.sim.Sim(nout=nout)
# Check for tree file before reading.
# Or, the fortran routine crashes causing the kernel crash without any error message. 

[sim._hilbert_cpulist] No AMR instance,
[sim._hilbert_cpulist] Loading one...
An AMR instance is created
 Sim
An AMR instance is created
 Sim
Simulation set up.


In [3]:
#no, ns, zr, ae = 
nnza = np.genfromtxt("./nout_nstep_zred_aexp.txt",
                     dtype=[("nout", int),
                            ("nstep", int),
                            ("zred", float),
                            ("aexp", float)])

In [ ]:
all_gcats=[]
for nout in nnza["nout"]:
    all_gcats.append(tree.halomodule.Halo(nout=nout, is_gal=True))

[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =./


In [30]:
gcat = tree.halomodule.Halo(nout=nout, is_gal=True)


[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =./


In [7]:
from galaxymodule import mk_gal_params as mgp
mgp.HAGN["verbose"] = False

In [8]:
tt = tree.tmtree.Tree(is_gal=True)

In [15]:
tnow = tt.tree[tt.tree["nstep"]==max(tt.tree["nstep"])]
large_last = tnow[(tnow["m"] > 1) * (tnow["m"] < 2)]
print(large_last["idx"])


[7864648 7864649 7864653 ..., 7985705 7986018 7986759]


In [16]:
final_idxs = large_last["idx"][large_last["id"] % 10 == 5]
final_ids = large_last["id"][large_last["id"] % 10 == 5]

In [80]:
for aexp in tt.aexps:
    print(nnza["nout"][np.abs(aexp - nnza["aexp"]).argmin()])

43
57
70
83
90
97
102
107
112
118
125
131
139
147
152
154
161
168
175
183
197
213
230
248
266
278
290
302
315
327
343
358
374
389
406
422
439
454
470
486
503
519
535
552
570
587
604
621
638
654
670
680
691
702
713
723
733
742
752
761
772
782


In [64]:
merger_dist_all=[]
for fid,fidx in zip(final_ids, final_idxs):
    maintree, idx_prgs_alltime = tt.extract_direct_full_tree(fidx)
    all_direct_prgs = tt.get_all_trees(idx_prgs_alltime)

    
    all_step=[]
    for this_step in all_direct_prgs[1:2]:
        step_now = 782
        gg = load.rd_GM.Gal(nout=step_now,
                            catalog=np.copy(gcat.data[fid-1]),
                            info=s.info)
        gg.debug=False
        #gg.mk_gal(**mgp.HAGN)
        mk_gal(gg,**mgp.HAGN)
        gg.cal_norm_vec()
        # Make sure that the catalog is not being modified.
        
        all_gal = []
        for this_gal in this_step:
            this_gal_relang=[]
            #this_gal_relang.append(0)
            if this_gal is None:
                continue
            for gal in this_gal[0:1]: 
                #only at the last moment
                #gal = this_gal[0]
                print(gal["nstep"])
                rel_pos = (gg.header["xg"] - gal["xp"])*1e3 # in kpc
                rel_vel = gg.header["vg"] - gal["vp"] 
                jx,jy,jz = np.cross(rel_pos, rel_vel)
                j_orbital=(jx,jy,jz)/np.sqrt(jx**2 + jy**2 + jz**2)
                # rotation axis
                relang = 180 / np.pi * np.arccos(np.dot(gg.meta.nvec, j_orbital))
                this_gal_relang.append(relang)
            all_gal.append(this_gal_relang)
        #print("---")
        all_step.append(all_gal)
        
    merger_dist_all.append(all_step)

0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
60-th step
61-th step
62-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000045
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000045
No CELL data loaded
gal.debug False
61
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step

/home/hoseung/Work/pyclusterevol/load/rd_GM.py:112: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  assert(not(idgal == None and catalog == None)), ("either idgal or a catalog"



9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
60-th step
61-th step
62-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000095
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000095
No CELL data loaded
gal.debug False
61
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step


3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
60-th step
61-th step
62-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000385
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000385
No CELL data loaded
gal.debug False
61
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th s

7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0000635
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0000635
No CELL data loaded
gal.debug False
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th 

...

17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0006245
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0006245
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th 

9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
58-th step
59-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0007085
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0007085
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th s

19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0008705
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0008705
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th 

4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
56-th step
57-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0014055
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0014055
No CELL data loaded
gal.debug False
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
1

4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th step
21-th step
22-th step
23-th step
24-th step
25-th step
26-th step
27-th step
28-th step
29-th step
30-th step
31-th step
32-th step
33-th step
34-th step
35-th step
36-th step
37-th step
38-th step
39-th step
40-th step
41-th step
42-th step
43-th step
44-th step
45-th step
46-th step
47-th step
48-th step
49-th step
50-th step
51-th step
52-th step
53-th step
54-th step
55-th step
info <load.info.Info object at 0x7f8a940b6fd0>
File Not Found: ./halo/HAL_00782/hal_dms_0115935
No DM data loaded
File Not Found: ./GalaxyMaker/CELL_00782/gal_cells_0115935
No CELL data loaded
gal.debug False
61
61
0-th step
1-th step
2-th step
3-th step
4-th step
5-th step
6-th step
7-th step
8-th step
9-th step
10-th step
11-th step
12-th step
13-th step
14-th step
15-th step
16-th step
17-th step
18-th step
19-th step
20-th ste

In [41]:
fig, ax = plt.subplots()
for all_step, idx in zip(merger_dist_all[2:3],final_idxs[2:3]):
    for i, this_step in enumerate(all_step):
        for all_gal in this_step:
            ax.scatter(t_lookback[i],all_gal, c=cm.hot(i*4))
            #for this_gal in all_gal:
                #print(this_gal)
            #print("--")
    ax.set_ylim([0,180])
    ax.set_title("Merger angle")
    ax.set_xlabel("lookback time")
    ax.set_ylabel("degree")
    plt.show()#savefig("Merger_angle_dist_"+str(idx).zfill(8)+".png")
    #plt.cla()

In [32]:
t_lookback = tt.age_univs[-1] - np.array(tt.age_univs[::-1])

In [45]:
from matplotlib import cm
fig, ax = plt.subplots()

for all_step, idx in zip(merger_dist_all,final_idxs):
    for i, this_step in enumerate(all_step):
        for all_gal in this_step:
            ax.scatter(t_lookback[i],all_gal, c=cm.hot(i*4))
            #for this_gal in all_gal:
                #print(this_gal)
            #print("--")
    ax.set_ylim([0,180])
    ax.set_title("Merger angle")
    ax.set_xlabel("lookback time")
    ax.set_ylabel("degree")
    plt.savefig("Merger_angle_dist_"+str(idx).zfill(8)+".png")
    plt.cla()

In [2]:
l = [item for sublist in merger_dist_all for item in sublist]
ll = [item for sublist in l for item in sublist]
lll = np.array(ll).ravel()

NameError: name 'merger_dist_all' is not defined

In [69]:
fig, ax = plt.subplots()
ax.hist(lll)
ax.set_title("relaive angle distribution (183 galaxies, 1~2e11M*, z=0)")
ax.set_xlabel("degree")
ax.set_ylabel("#")
plt.show()


# B/T 

In [70]:
gg.cal_b2t()

TypeError: 'NoneType' object is not subscriptable

In [18]:
all_step=[]
for this_step in all_data[1:]:
    all_gal = []
    for this_gal in this_step:
        this_gal_relang=[]
        #this_gal_relang.append(0)
        if this_gal is None:
            continue
        for gal in this_gal[0:1]: #only at the last moment
            #gal = this_gal[0]
            #print(gal)
            rel_pos = (gg.header["xg"] - gal["xp"])*1e3 # in kpc
            rel_vel = gg.header["vg"] - gal["vp"] 
            jx,jy,jz = np.cross(rel_pos, rel_vel)
            j_orbital=(jx,jy,jz)/np.sqrt(jx**2 + jy**2 + jz**2)
            # rotation axis
            relang = 180 / np.pi * np.arccos(np.dot(gg.meta.nvec, j_orbital))
            this_gal_relang.append(relang)
        all_gal.append(this_gal_relang)
    #print("---")
    all_step.append(all_gal)